# Labelme转COCO-单个文件

同济子豪兄 呕心沥血写成

兼容目标检测、图像分割、关键点检测三种标注

2023-3-9

2023-3-10

2023-4-15

2023-4-16

2023-4-21

## 导入工具包

In [1]:
import os
import json
import numpy as np

## 删除系统自动生成的多余文件

建议在 Linux 系统中运行爬虫、划分训练集测试集代码

### 查看待删除的多余文件

In [2]:
!find . -iname '__MACOSX'

In [3]:
!find . -iname '.DS_Store'

In [4]:
!find . -iname '.ipynb_checkpoints'

./output_coco/.ipynb_checkpoints
./.ipynb_checkpoints


### 删除多余文件

In [5]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [6]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [7]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [8]:
!find . -iname '__MACOSX'

In [9]:
!find . -iname '.DS_Store'

In [10]:
!find . -iname '.ipynb_checkpoints'

## 载入一个labelme格式的json标注文件

In [11]:
with open('1.json', 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [12]:
labelme.keys()

dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth'])

## 元数据 

In [13]:
labelme['version']

'5.1.1'

In [14]:
labelme['flags']

{}

In [15]:
# 图像文件名
labelme['imagePath']

'1.jpg'

In [16]:
labelme['imageData']

In [17]:
# 图像高度
labelme['imageHeight']

2736

In [18]:
# 图像宽度
labelme['imageWidth']

3648

## 标注信息

In [19]:
labelme['shapes']

[{'label': 'Triangle_30',
  'points': [[172.52941176470614, 100.35294117647076],
   [2054.8823529411766, 976.8235294117648]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'Triangle_45',
  'points': [[2422.529411764706, 432.70588235294133],
   [3569.5882352941176, 1673.8823529411766]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'Angle',
  'points': [[1128.4117647058827, 1000.3529411764706],
   [2510.764705882353, 1844.4705882352944]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'Ruler',
  'points': [[290.17647058823553, 2029.764705882353],
   [2510.764705882353, 2626.8235294117644]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'Triangle_30_poly',
  'points': [[2040.1764705882354, 735.6470588235295],
   [566.6470588235296, 106.23529411764723],
   [181.3529411764708, 947.4117647058824]],
  'group_id': None,
  'shape_type': 'polygon',
  'flags': {}},
 {'label': 'T

## 创建coco格式的字典

In [20]:
coco = {}

## info

In [21]:
# coco['info'] = {}
# coco['info']['description'] = 'Labelme2coco keypoint format script from Zihao'
# coco['info']['year'] = 2023
# coco['info']['date_created'] = '2023/03/09'

## 类别

In [22]:
class_list= [
    {'id': 0, 'name': 'Triangle_30'},
    {'id': 1, 'name': 'Triangle_45'},
    {'id': 2, 'name': 'Angle'},
    {'id': 3, 'name': 'Ruler'}
]

In [23]:
label2id = {}
for each in class_list:
    label2id[each['name']] = each['id']

In [24]:
label2id

{'Triangle_30': 0, 'Triangle_45': 1, 'Angle': 2, 'Ruler': 3}

In [25]:
coco['categories'] = class_list
coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

## 函数-处理单个labelme标注json文件

In [27]:
def process_single_json(labelme, image_id=1):
    '''
    输入labelme的json数据，输出coco格式的每个框的关键点标注信息
    '''
    
    global ANN_ID
    
    coco_annotations = []
    
    for each_ann in labelme['shapes']: # 遍历该json文件中的所有标注

        if each_ann['shape_type'] == 'rectangle': # 筛选出框

            ## 该框的元数据
            bbox_dict = {}
            # 该框的类别 信息
            bbox_dict['category_id'] = label2id[each_ann['label']]
            bbox_dict['segmentation'] = []
            
            bbox_dict['iscrowd'] = 0
            bbox_dict['image_id'] = image_id
            bbox_dict['id'] = ANN_ID
            ANN_ID += 1

            # 获取框坐标
            bbox_left_top_x = min(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_left_top_y = min(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_right_bottom_x = max(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_right_bottom_y = max(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_w = bbox_right_bottom_x - bbox_left_top_x
            bbox_h = bbox_right_bottom_y - bbox_left_top_y
            bbox_dict['bbox'] = [bbox_left_top_x, bbox_left_top_y, bbox_w, bbox_h] # 左上角x、y、框的w、h
            bbox_dict['area'] = bbox_w * bbox_h
            
            # 筛选出分割多段线
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'polygon': # 筛选出分割多段线标注
                    # 第一个点的坐标
                    first_x = each_ann['points'][0][0]
                    first_y = each_ann['points'][0][1]
                    if (first_x>bbox_left_top_x) & (first_x<bbox_right_bottom_x) & (first_y<bbox_right_bottom_y) & (first_y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_dict['segmentation'] = list(map(lambda x: list(map(lambda y: round(y, 2), x)), each_ann['points'])) # 坐标保留两位小数
                    
            coco_annotations.append(bbox_dict)
            
    return coco_annotations

## 测试一下函数的效果，处理单个labelme格式的json标注文件

In [28]:
labelme_json_path = '1.json'

with open(labelme_json_path, 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [29]:
process_single_json(labelme)

[{'category_id': 0,
  'segmentation': [[2040.18, 735.65], [566.65, 106.24], [181.35, 947.41]],
  'iscrowd': 0,
  'image_id': 1,
  'id': 0,
  'bbox': [172, 100, 1882, 876],
  'area': 1648632},
 {'category_id': 1,
  'segmentation': [[3504.88, 1653.29], [3557.82, 523.88], [2431.35, 450.35]],
  'iscrowd': 0,
  'image_id': 1,
  'id': 1,
  'bbox': [2422, 432, 1147, 1241],
  'area': 1423427},
 {'category_id': 2,
  'segmentation': [[1151.94, 1697.41],
   [1166.65, 1544.47],
   [1210.76, 1388.59],
   [1310.76, 1247.41],
   [1443.12, 1132.71],
   [1599.0, 1050.35],
   [1772.53, 1018.0],
   [1969.59, 1023.88],
   [2113.71, 1076.82],
   [2257.82, 1162.12],
   [2381.35, 1297.41],
   [2443.12, 1418.0],
   [2490.18, 1579.76],
   [2499.0, 1738.59],
   [2475.47, 1832.71]],
  'iscrowd': 0,
  'image_id': 1,
  'id': 2,
  'bbox': [1128, 1000, 1382, 844],
  'area': 1166408},
 {'category_id': 3,
  'segmentation': [[343.12, 2047.41],
   [313.71, 2444.47],
   [2478.41, 2597.41],
   [2496.06, 2197.41]],
  'iscr

## images和annotations

In [30]:
IMG_ID = 0
ANN_ID = 0

In [31]:
# 遍历所有 labelme 格式的 json 文件
for labelme_json in os.listdir(): 
    
    if labelme_json.split('.')[-1] == 'json':
        
        with open(labelme_json, 'r', encoding='utf-8') as f:
            
            labelme = json.load(f)
            
            ## 提取图像元数据
            img_dict = {}
            img_dict['file_name'] = labelme['imagePath']
            img_dict['height'] = labelme['imageHeight']
            img_dict['width'] = labelme['imageWidth']
            img_dict['id'] = IMG_ID
            coco['images'].append(img_dict)
            
            ## 提取框和关键点信息
            coco_annotations = process_single_json(labelme, image_id=IMG_ID)
            coco['annotations'] += coco_annotations
            
            IMG_ID += 1
            
            print(labelme_json, '已处理完毕')

    else:
        pass

1.json 已处理完毕


## 保存为MS COCO格式的json标注文件

In [32]:
if not os.path.exists('output_coco'):
    os.mkdir('output_coco')
    print('创建新目录 output_coco')

In [33]:
coco_path = 'output_coco/coco_sample.json'

In [34]:
with open(coco_path, 'w') as f:
    json.dump(coco, f, indent=2)

In [35]:
coco

{'categories': [{'id': 0, 'name': 'Triangle_30'},
  {'id': 1, 'name': 'Triangle_45'},
  {'id': 2, 'name': 'Angle'},
  {'id': 3, 'name': 'Ruler'}],
 'images': [{'file_name': '1.jpg', 'height': 2736, 'width': 3648, 'id': 0}],
 'annotations': [{'category_id': 0,
   'segmentation': [[2040.18, 735.65], [566.65, 106.24], [181.35, 947.41]],
   'iscrowd': 0,
   'image_id': 0,
   'id': 0,
   'bbox': [172, 100, 1882, 876],
   'area': 1648632},
  {'category_id': 1,
   'segmentation': [[3504.88, 1653.29], [3557.82, 523.88], [2431.35, 450.35]],
   'iscrowd': 0,
   'image_id': 0,
   'id': 1,
   'bbox': [2422, 432, 1147, 1241],
   'area': 1423427},
  {'category_id': 2,
   'segmentation': [[1151.94, 1697.41],
    [1166.65, 1544.47],
    [1210.76, 1388.59],
    [1310.76, 1247.41],
    [1443.12, 1132.71],
    [1599.0, 1050.35],
    [1772.53, 1018.0],
    [1969.59, 1023.88],
    [2113.71, 1076.82],
    [2257.82, 1162.12],
    [2381.35, 1297.41],
    [2443.12, 1418.0],
    [2490.18, 1579.76],
    [2499.0,

## 验证MS COCO格式的标注

In [36]:
# !pip install pycocotools

In [37]:
from pycocotools.coco import COCO

my_coco = COCO(coco_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
